本周只有一个代码实践题

以下代码课用来加载 预先训练好的模型,你需要只需要修改模型的存放路径即可（第二行代码）

In [5]:
!pip install transformers


Looking in indexes: https://pypi.douban.com/simple


In [1]:
import torch
import torch.nn.functional as F
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config
from torch.utils.data import DataLoader,Dataset

第一步 使用以下链接下载相应预训练模型

https://convaisharables.blob.core.windows.net/lsp/multiref/small_ft.pkl

In [5]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
weights = torch.load('../Dataset/Assignment_13/small_ft.pkl')
medium_config = GPT2Config(n_embd = 768,n_layer = 12, n_head = 12)
model = GPT2LMHeadModel(medium_config)

weights['lm_head.weight'] = weights['lm_head.decoder.weight']
weights.pop('lm_head.decoder.weight',None)

model.load_state_dict(weights)
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

你需要写一个推理函数，这个函数接收一个英文句子为输入，输出一个回应。

In [60]:
def qa(question):
    
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    tokens = question.strip()
    
    def get_word(token):
        with torch.no_grad():
            indexed_token = tokenizer.encode(token)
            token_tensor = torch.tensor([indexed_token])
            outputs = model(token_tensor)
            predictions = outputs[0]
            predicted_index = torch.argmax(predictions[0,-1,:]).item()
            predicted_word = tokenizer.decode([predicted_index])
            return predicted_word
    
    if tokens is '':
        return "Input question is null"
    else:
        answer = ""
        answer_word = get_word(tokens)
        i = 0
        while(i<2):
            if(answer_word=='<|endoftext|>'):
                i+=1
            else:
                answer = answer +  answer_word
                tokens = tokens +  answer_word
            answer_word = get_word(tokens)

        return answer

### 测试语句 
一轮对话   
1. Does money buy happiness ?   
2. What is the best way to buy happiness?   

一轮对话   
1. what is the meaning of a godd life ?   
2. How to be a good person ?   

In [62]:
qa("Does money buy happiness ?")

'... maybe happiness bought by happiness?'

In [63]:
qa("What is the best way to buy happiness?")

' Buy happiness.sell'

In [69]:
qa("what is the meaning of a good life ?")

' Citizelessness'

In [65]:
qa("How to be a good person ?")

' we are all guilty sinners.gif'